In [1]:
from ydata_profiling import ProfileReport
import pandas as pd
import glob
import os

In [2]:
server_traffic_folder = "data"
daily_traffic_files = glob.glob(os.path.join(server_traffic_folder, "*.csv"))

In [3]:
daily_traffic_list = [pd.read_csv(daily_traffic_file, encoding="ISO-8859-1") for daily_traffic_file in daily_traffic_files]
weekly_traffic_df = pd.concat(daily_traffic_list, axis=0)
weekly_traffic_df

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.14-209.48.71.168-49459-80-6,192.168.10.14,49459,209.48.71.168,80,6,5/7/2017 8:42,38308,1,1,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,192.168.10.3-192.168.10.17-389-49453-6,192.168.10.17,49453,192.168.10.3,389,6,5/7/2017 8:42,479,11,5,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,192.168.10.3-192.168.10.17-88-46124-6,192.168.10.17,46124,192.168.10.3,88,6,5/7/2017 8:42,1095,10,6,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,192.168.10.3-192.168.10.17-389-49454-6,192.168.10.17,49454,192.168.10.3,389,6,5/7/2017 8:42,15206,17,12,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,192.168.10.3-192.168.10.17-88-46126-6,192.168.10.17,46126,192.168.10.3,88,6,5/7/2017 8:42,1092,9,6,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170324,157.240.18.35-192.168.10.51-443-55641-6,157.240.18.35,443,192.168.10.51,55641,6,6/7/2017 12:59,49,1,3,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
170325,192.168.10.51-199.16.156.120-45337-443-6,199.16.156.120,443,192.168.10.51,45337,6,6/7/2017 12:59,217,2,1,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
170326,192.168.10.12-192.168.10.50-60148-22-6,192.168.10.12,60148,192.168.10.50,22,6,6/7/2017 12:59,1387547,41,46,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
170327,192.168.10.12-192.168.10.50-60146-22-6,192.168.10.12,60146,192.168.10.50,22,6,6/7/2017 12:59,207,1,1,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [4]:
filename = 'Weekly-WorkingHours_report'

In [5]:
weekly_traffic_report_filename = os.path.join('data', filename + '.csv')
weekly_traffic_df.to_csv(weekly_traffic_report_filename, index=False)

In [6]:
type_schema = {' Label': "categorical"}

In [7]:
weekly_traffic_profile = ProfileReport(weekly_traffic_df, title="Server Traffic Profiling Report", minimal=True, type_schema=type_schema)
weekly_traffic_profiling_report_filename = os.path.join('data', filename + '.html')
weekly_traffic_profile.to_file(weekly_traffic_profiling_report_filename)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/cesarealice/PycharmProjects/generative-ml-malicious-traffic-wolf-in-sheeps-clothes/.venv/lib/python3.10/site-packages/numpy/_core/_methods.py:194: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/cesarealice/PycharmProjects/generative-ml-malicious-traffic-wolf-in-sheeps-clothes/.venv/lib/python3.10/site-packages/numpy/_core/_methods.py:194: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/cesarealice/PycharmProjects/generative-ml-malicious-traffic-wolf-in-sheeps-clothes/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1344: RuntimeWarning: invalid value encountered in subtract
  adjusted = values - mean
/home/cesarealice/PycharmProjects/generative-ml-malicious-traffic-wolf-in-sheeps-clothes/.venv/lib/python3.10/site-packages/numpy/_core/_methods.py:194: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/cesarealice/PycharmProjects/generative-ml-ma

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
weekly_traffic_df.columns

Index(['Flow ID', ' Source IP', ' Source Port', ' Destination IP',
       ' Destination Port', ' Protocol', ' Timestamp', ' Flow Duration',
       ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Length', ' Max Pa

In [9]:
weekly_traffic_df[' Label'].unique()

array(['BENIGN', 'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk',
       'DoS GoldenEye', 'Heartbleed', 'Infiltration', 'PortScan',
       'FTP-Patator', 'SSH-Patator', 'DDoS', 'Bot',
       'Web Attack Â\x96 Brute Force', 'Web Attack Â\x96 XSS',
       'Web Attack Â\x96 Sql Injection', 'Web Attack \x96 Brute Force',
       'Web Attack \x96 XSS', 'Web Attack \x96 Sql Injection'],
      dtype=object)

In [10]:
VALID_TRAFFIC_TYPES = [
    'BENIGN',
    'DoS Hulk',
    'PortScan',
    'DDoS',
    'DoS GoldenEye',
    'FTP-Patator'
]
weekly_traffic_valid_traffic_df = weekly_traffic_df.loc[weekly_traffic_df[' Label'].isin(VALID_TRAFFIC_TYPES)]
weekly_traffic_valid_traffic_df

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.14-209.48.71.168-49459-80-6,192.168.10.14,49459,209.48.71.168,80,6,5/7/2017 8:42,38308,1,1,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,192.168.10.3-192.168.10.17-389-49453-6,192.168.10.17,49453,192.168.10.3,389,6,5/7/2017 8:42,479,11,5,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,192.168.10.3-192.168.10.17-88-46124-6,192.168.10.17,46124,192.168.10.3,88,6,5/7/2017 8:42,1095,10,6,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,192.168.10.3-192.168.10.17-389-49454-6,192.168.10.17,49454,192.168.10.3,389,6,5/7/2017 8:42,15206,17,12,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,192.168.10.3-192.168.10.17-88-46126-6,192.168.10.17,46126,192.168.10.3,88,6,5/7/2017 8:42,1092,9,6,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170324,157.240.18.35-192.168.10.51-443-55641-6,157.240.18.35,443,192.168.10.51,55641,6,6/7/2017 12:59,49,1,3,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
170325,192.168.10.51-199.16.156.120-45337-443-6,199.16.156.120,443,192.168.10.51,45337,6,6/7/2017 12:59,217,2,1,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
170326,192.168.10.12-192.168.10.50-60148-22-6,192.168.10.12,60148,192.168.10.50,22,6,6/7/2017 12:59,1387547,41,46,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
170327,192.168.10.12-192.168.10.50-60146-22-6,192.168.10.12,60146,192.168.10.50,22,6,6/7/2017 12:59,207,1,1,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [11]:
weekly_traffic_valid_traffic_df.columns

Index(['Flow ID', ' Source IP', ' Source Port', ' Destination IP',
       ' Destination Port', ' Protocol', ' Timestamp', ' Flow Duration',
       ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Length', ' Max Pa

In [12]:
weekly_traffic_valid_traffic_profile = ProfileReport(weekly_traffic_valid_traffic_df, title="Server Traffic Profiling Report", minimal=True, type_schema=type_schema)
weekly_traffic_valid_traffic_profiling_report_filename = os.path.join('data', filename + '_valid_traffic.html')
weekly_traffic_valid_traffic_profile.to_file(weekly_traffic_valid_traffic_profiling_report_filename)

/home/cesarealice/PycharmProjects/generative-ml-malicious-traffic-wolf-in-sheeps-clothes/.venv/lib/python3.10/site-packages/ydata_profiling/utils/dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/cesarealice/PycharmProjects/generative-ml-malicious-traffic-wolf-in-sheeps-clothes/.venv/lib/python3.10/site-packages/numpy/_core/_methods.py:194: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/cesarealice/PycharmProjects/generative-ml-malicious-traffic-wolf-in-sheeps-clothes/.venv/lib/python3.10/site-packages/numpy/_core/_methods.py:194: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/cesarealice/PycharmProjects/generative-ml-malicious-traffic-wolf-in-sheeps-clothes/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1344: RuntimeWarning: invalid value encountered in subtract
  adjusted = values - mean
/home/cesarealice/PycharmProjects/generative-ml-malicious-traffic-wolf-in-sheeps-clothes/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1256: RuntimeWarning: invalid value encountered in subtract
  adjusted = values - mean
/home/cesarealice/PycharmProjects/generative-ml-maliciou

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
# Apply Down sampling to solve the unbalanced
n_instances_per_traffic_type = 150000
weekly_traffic_valid_traffic_list = []
for valid_traffic_type in VALID_TRAFFIC_TYPES:
    # filter by type and sample
    traffic_type_df = weekly_traffic_valid_traffic_df.loc[weekly_traffic_valid_traffic_df[' Label'] == valid_traffic_type]
    # check if there are enough instances to apply sampling
    if traffic_type_df.shape[0]>n_instances_per_traffic_type:
        traffic_type_df = traffic_type_df.sample(n_instances_per_traffic_type)
    # concatenate result
    weekly_traffic_valid_traffic_list.append(traffic_type_df)

# concatenate the results
weekly_traffic_valid_traffic_balanced_df = pd.concat(weekly_traffic_valid_traffic_list, axis=0)
weekly_traffic_valid_traffic_balanced_df

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
169830,192.168.10.25-52.71.109.106-53399-443-6,192.168.10.25,53399,52.71.109.106,443,6,03/07/2017 12:20:32,49520,5,0,...,32,0.0000,0.0000,0.0,0.0,0.000000e+00,0.00000,0.0,0.0,BENIGN
334804,192.168.10.9-34.225.51.157-11434-443-6,34.225.51.157,443,192.168.10.9,11434,6,4/7/2017 4:27,14,1,1,...,20,0.0000,0.0000,0.0,0.0,0.000000e+00,0.00000,0.0,0.0,BENIGN
5331735,192.168.10.3-192.168.10.19-53-26108-17,192.168.10.19,26108,192.168.10.3,53,17,6/7/2017 11:59,94113,2,2,...,32,0.0000,0.0000,0.0,0.0,0.000000e+00,0.00000,0.0,0.0,BENIGN
4620221,192.168.10.1-192.168.10.3-53-60661-17,192.168.10.3,60661,192.168.10.1,53,17,03/07/2017 02:20:20,39718603,2,2,...,20,46645.0000,0.0000,46645.0,46645.0,3.962204e+07,0.00000,39622045.0,39622045.0,BENIGN
191830,192.168.10.14-38.69.238.16-55762-80-6,192.168.10.14,55762,38.69.238.16,80,6,4/7/2017 2:41,119061659,16,14,...,20,316570.1818,943646.3737,3161771.0,31902.0,9.993783e+06,19973.45353,10000000.0,9933573.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4090129,172.16.0.1-192.168.10.50-52114-21-6,172.16.0.1,52114,192.168.10.50,21,6,4/7/2017 9:19,9794876,9,15,...,32,0.0000,0.0000,0.0,0.0,0.000000e+00,0.00000,0.0,0.0,FTP-Patator
4090130,172.16.0.1-192.168.10.50-52126-21-6,172.16.0.1,52126,192.168.10.50,21,6,4/7/2017 9:19,9802316,9,15,...,32,0.0000,0.0000,0.0,0.0,0.000000e+00,0.00000,0.0,0.0,FTP-Patator
4090131,172.16.0.1-192.168.10.50-52118-21-6,172.16.0.1,52118,192.168.10.50,21,6,4/7/2017 9:19,9803729,9,15,...,32,0.0000,0.0000,0.0,0.0,0.000000e+00,0.00000,0.0,0.0,FTP-Patator
4090133,172.16.0.1-192.168.10.50-52130-21-6,172.16.0.1,52130,192.168.10.50,21,6,4/7/2017 9:19,9811139,9,15,...,32,0.0000,0.0000,0.0,0.0,0.000000e+00,0.00000,0.0,0.0,FTP-Patator


In [14]:
weekly_traffic_valid_traffic_balanced_profile = ProfileReport(weekly_traffic_valid_traffic_balanced_df, title="Server Traffic Profiling Report", minimal=True, type_schema=type_schema)
weekly_traffic_valid_traffic_balanced_profiling_report_filename = os.path.join('data', filename + '_valid_traffic_balanced.html')
weekly_traffic_valid_traffic_balanced_profile.to_file(weekly_traffic_valid_traffic_balanced_profiling_report_filename)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/cesarealice/PycharmProjects/generative-ml-malicious-traffic-wolf-in-sheeps-clothes/.venv/lib/python3.10/site-packages/numpy/_core/_methods.py:194: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/cesarealice/PycharmProjects/generative-ml-malicious-traffic-wolf-in-sheeps-clothes/.venv/lib/python3.10/site-packages/numpy/_core/_methods.py:194: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/cesarealice/PycharmProjects/generative-ml-malicious-traffic-wolf-in-sheeps-clothes/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1344: RuntimeWarning: invalid value encountered in subtract
  adjusted = values - mean
/home/cesarealice/PycharmProjects/generative-ml-malicious-traffic-wolf-in-sheeps-clothes/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1256: RuntimeWarning: invalid value encountered in subtract
  adjusted = values - mean
/home/cesarealice/PycharmProjects/generative-ml-maliciou

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
weekly_traffic_valid_traffic_balanced_df[weekly_traffic_valid_traffic_balanced_df['Flow ID'].duplicated()].shape

(379194, 85)

In [16]:
weekly_traffic_df.duplicated().shape

(8182756,)

In [17]:
weekly_traffic_df.loc[weekly_traffic_df['Flow ID'] =='178.217.128.81-192.168.10.16-80-37474-6']

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
5234041,178.217.128.81-192.168.10.16-80-37474-6,192.168.10.16,37474,178.217.128.81,80,6,6/7/2017 9:51,117944143,64,71,...,32,400649.3636,838807.4877,2929749.0,147526.0,10100000.0,7038.26627,10100000.0,10100000.0,BENIGN
5236858,178.217.128.81-192.168.10.16-80-37474-6,178.217.128.81,80,192.168.10.16,37474,6,6/7/2017 9:53,49,1,1,...,32,0.0000,0.0000,0.0,0.0,0.0,0.00000,0.0,0.0,BENIGN
52320,178.217.128.81-192.168.10.16-80-37474-6,192.168.10.16,37474,178.217.128.81,80,6,6/7/2017 9:51,117944143,64,71,...,32,400649.3636,838807.4877,2929749.0,147526.0,10100000.0,7038.26627,10100000.0,10100000.0,BENIGN
55137,178.217.128.81-192.168.10.16-80-37474-6,178.217.128.81,80,192.168.10.16,37474,6,6/7/2017 9:53,49,1,1,...,32,0.0000,0.0000,0.0,0.0,0.0,0.00000,0.0,0.0,BENIGN


In [23]:
# Remove irrelevant features
from utils.constants import RELEVANT_COLUMNS, VALID_PORT_RANGE, VALID_PROTOCOL_VALUES, VALID_TRAFFIC_TYPES
from pipeline.preprocess.preprocess import preprocessing
base_traffic_df = weekly_traffic_valid_traffic_balanced_df[RELEVANT_COLUMNS]

Non-informative
* Flow ID
* Source IP
* Destination IP
* Timestamp

Low Variance
* Fwd PSH Flags
* Fwd URG Flags
* FIN Flag Count
* SYN Flag Count
* RST Flag Count
* URG Flag Count
* CWE Flag Count
* ECE Flag Count

Single Value
* Bwd PSH Flags
* Bwd URG Flags
* Fwd Avg Bytes/Bulk
* Fwd Avg Packets/Bulk
* Fwd Avg Bulk Rate
* Bwd Avg Bytes/Bulk
* Bwd Avg Packets/Bulk
* Bwd Avg Bulk Rate

Duplicated feature
* Fwd Header Length.1

In [19]:
base_traffic_profile = ProfileReport(base_traffic_df, title="Server Traffic Profiling Report", minimal=True, type_schema=type_schema)
base_traffic_filename = os.path.join('data', filename + '_base1.html')
base_traffic_profile.to_file(base_traffic_filename)

/home/cesarealice/PycharmProjects/generative-ml-malicious-traffic-wolf-in-sheeps-clothes/.venv/lib/python3.10/site-packages/ydata_profiling/utils/dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/cesarealice/PycharmProjects/generative-ml-malicious-traffic-wolf-in-sheeps-clothes/.venv/lib/python3.10/site-packages/numpy/_core/_methods.py:194: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/cesarealice/PycharmProjects/generative-ml-malicious-traffic-wolf-in-sheeps-clothes/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1344: RuntimeWarning: invalid value encountered in subtract
  adjusted = values - mean
/home/cesarealice/PycharmProjects/generative-ml-malicious-traffic-wolf-in-sheeps-clothes/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1256: RuntimeWarning: invalid value encountered in subtract
  adjusted = values - mean
/home/cesarealice/PycharmProjects/generative-ml-malicious-traffic-wolf-in-sheeps-clothes/.venv/lib/python3.10/site-packages/numpy/_core/_methods.py:194: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/cesarealice/PycharmProjects/generative-ml-maliciou

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
# save in csv file
base_traffic_filepath = 'data/Weekly-WorkingHours_report_base1.csv'
base_traffic_df.to_csv(base_traffic_filepath, index=False)

base_traffic_cleaned_df = preprocessing(base_traffic_filepath, relevant_column=RELEVANT_COLUMNS, valid_traffic_types=VALID_TRAFFIC_TYPES,
                                        valid_port_range=VALID_PORT_RANGE, valid_protocol_values=VALID_PROTOCOL_VALUES)
base_traffic_cleaned_df

data cleaning
valid labels due frequency: ['BENIGN', 'DoS Hulk', 'PortScan', 'DDoS', 'DoS GoldenEye', 'FTP-Patator']


KeyError: "None of [Index(['BENIGN', 'DoS Hulk', 'PortScan', 'DDoS', 'DoS GoldenEye',\n       'FTP-Patator'],\n      dtype='object')] are in the [columns]"

In [ ]:
base_traffic_cleaned_profile = ProfileReport(base_traffic_cleaned_df, title="Server Traffic Profiling Report", minimal=True, type_schema=type_schema)
base_traffic_cleaned_filename = os.path.join('data', filename + '_base_cleaned.html')
base_traffic_cleaned_profile.to_file(base_traffic_cleaned_filename)